In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from scipy.special import erfc
from IPython.display import HTML, display
import math

%matplotlib notebook


In [ ]:
n = 50
dx = 1.0 / n
dt = (dx**2) / 4
D = 1.0
x = np.linspace(0, 1, n+1)
y = np.linspace(0, 1, n+1)

print(f"dt = {dt}, Stability check: {dt * D / (dx**2)}")

In [ ]:
def next_step(c, dt=dt, dx=dx, D=D):
    cons = dt * D / (dx**2)
    
    if cons > 1:
        raise ValueError("Time step too large! The scheme is unstable.")
    
    n = len(c)
    c_new = np.copy(c)

    for j in range(1, n-1):
        # West boundary (x = 0)
        c_new[j, 0] = c[j, 0] + cons * (
            c[j, 1] + c[j, -2] + c[j+1, 0] + c[j-1, 0] - 4 * c[j, 0]
        )

        # Interior points
        for i in range(1, n-1):
            c_new[j, i] = c[j, i] + cons * (
                c[j, i+1] + c[j, i-1] + c[j+1, i] + c[j-1, i] - 4 * c[j, i]
            )

        # East boundary (x = n)
        c_new[j, -1] = c[j, -1] + cons * (
            c[j, 1] + c[j, -2] + c[j+1, -1] + c[j-1, -1] - 4 * c[j, -1]
        )


    return c_new


In [ ]:
c = np.zeros((n, n))
c[0, :] = 1  # top boundary
c[-1, :] = 0  # bottom boundary


fig, ax = plt.subplots()
im = ax.imshow(c, cmap="hot", animated=True)

ax.set_xlabel(fr"Intervals $\delta x$ ({n} here)")
ax.set_ylabel(fr"Intervals $\delta y$ ({n} here)")
ax.set_title("Concentration Diffusion Over Time")

def update(frame):
    global c
    c = next_step(c)
    im.set_array(c)
    return [im]

ani = animation.FuncAnimation(fig, update, frames=1000, interval=50, blit=False)
display(HTML(ani.to_jshtml()))


In [ ]:
%matplotlib inline


plt.figure(figsize=(8, 6))

# numerical 
for idx, _ in enumerate(frames_to_save):
    plt.plot(y_vals, c_y[idx], label=f"Simulation (t={dt_values[idx]})", linestyle="-", marker="o", markersize=3)

# analytical
for t_idx, t in enumerate(frames_to_save):
    c_analytic = []
    for y in y_vals:
        c_t = sum([
            erfc((1 - y + 2 * i) / (2 * np.sqrt(D * dt_values[t_idx]))) - 
            erfc((1 + y + 2 * i) / (2 * np.sqrt(D * dt_values[t_idx])))
            for i in range(50) # for 50 terms, can change
        ])
        c_analytic.append(c_t)

    plt.plot(y_vals, c_analytic, label=f"Analytical (t={dt_values[t_idx]})", linestyle="dashed")

plt.xlabel("y values")
plt.ylabel("c(y) values")
plt.title("Comparison: Simulation vs. Analytical")
plt.legend()
plt.show()
